### Libraries

In [1]:
import torch #type:ignore
from torch import nn #type:ignore
from torch.utils.data import DataLoader #type:ignore
from torchvision import datasets #type:ignore
from torchvision.transforms import ToTensor #type:ignore

#### Download Training data

In [2]:

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

#### Data Loaders

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


#### Select Device

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


#### Define Model

In [5]:

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


#### Optimizer

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

#### Train Model

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

#### Test Model

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#### Training Process and Testing Process Initialize

In [15]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.147289  [   64/60000]
loss: 1.166810  [ 6464/60000]
loss: 0.998967  [12864/60000]
loss: 1.148355  [19264/60000]
loss: 1.037952  [25664/60000]
loss: 1.081990  [32064/60000]
loss: 1.123477  [38464/60000]
loss: 1.051639  [44864/60000]
loss: 1.114834  [51264/60000]
loss: 1.057385  [57664/60000]
Test Error: 
 Accuracy: 64.5%, Avg loss: 1.082358 

Epoch 2
-------------------------------
loss: 1.147289  [   64/60000]
loss: 1.166810  [ 6464/60000]
loss: 0.998967  [12864/60000]
loss: 1.148355  [19264/60000]
loss: 1.037952  [25664/60000]
loss: 1.081990  [32064/60000]
loss: 1.123477  [38464/60000]
loss: 1.051639  [44864/60000]
loss: 1.114834  [51264/60000]
loss: 1.057385  [57664/60000]
Test Error: 
 Accuracy: 64.5%, Avg loss: 1.082358 

Epoch 3
-------------------------------
loss: 1.147289  [   64/60000]
loss: 1.166810  [ 6464/60000]
loss: 0.998967  [12864/60000]
loss: 1.148355  [19264/60000]
loss: 1.037952  [25664/60000]
loss: 1.081990  [32064/600

#### Save Model 

In [16]:
torch.save(model.state_dict(), "Fashion_Minist_model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


#### Loading Model

In [17]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("Fashion_Minist_model.pth", weights_only=True))

<All keys matched successfully>

#### Perdict Output

In [18]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[66][0], test_data[66][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Dress", Actual: "Pullover"
